## Ingesting raw UNODC Data

In [ ]:
import requests # Collecting payloads from URLs
import os # Great for navigating inside your file system


In [ ]:
# Defining root and raw for ingestion
project_root = os.path.dirname(os.getcwd())
data_root = os.path.join(project_root, 'data')
raw_path = os.path.join(data_root, 'raw')
print(data_root)
print(raw_path)

In [ ]:
# Since we have a complete data dump, we can overwrite it every time
def get_file_data(url):
    file_name = url.split('/')[-1] # Gets file name
    folder_name = os.path.join(raw_path, file_name.split('.')[0]) # Removing the file extension for folder name
    response = requests.get(url)
    
    # Creating and writing the files to directory
    os.makedirs(folder_name, exist_ok=True) 
    file_path = os.path.join(folder_name, file_name) 
    with open(file_path, 'wb') as f:
        f.write(response.content)

    print(f'extracted file to {folder_name}')

In [ ]:
# 3 Files from UNODC based on their classification
file_urls = ['https://dataunodc.un.org/sites/dataunodc.un.org/files/data_cts_violent_and_sexual_crime.xlsx',
             'https://dataunodc.un.org/sites/dataunodc.un.org/files/data_cts_access_and_functioning_of_justice.xlsx',
             'https://dataunodc.un.org/sites/dataunodc.un.org/files/data_cts_corruption_and_economic_crime.xlsx']

In [ ]:
# Let's go
for url in file_urls:
        get_file_data(url)

## Raw to bronze processing
- Info.txt has the nature of raw to bronze transformations
- We'll go through it using spark and pandas side by side

In [1]:
import os
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# from utils.common_utils import *

In [2]:
# We first need a SparkSession
# This is the entrypoint of any spark application

spark = SparkSession.builder.appName("raw_to_bronze_unodc").getOrCreate()

In [3]:
# config.yml has all the paths that we need
def load_config(config_path):
    with open(config_path, 'r') as file:
        return yaml.safe_load(file)
    
config_path = r'D:\python_projects\truth_machine\config.yaml'.replace("\\", "/")
config = load_config(config_path)

In [4]:
# Pandas is the best for reading excel since it deals with weird formatting better
# Spark isn't very good with varying schema because it's meant for huge data
# It will be spark.read.excel(file_path) in pyspark
df_func_justice = pd.read_excel(config['raw_paths']['unodc_func_justice'], engine='openpyxl', header=2)
df_econ_crime = pd.read_excel(config['raw_paths']['unodc_econ_crime'], engine='openpyxl', header=2)
df_violent_sexual = pd.read_excel(config['raw_paths']['unodc_sexual_violent'], engine='openpyxl', header=2)

In [5]:
# Converting between spark and pandas is easy as balls
df_func_justice_spark = spark.createDataFrame(df_func_justice)
df_econ_crime_spark = spark.createDataFrame(df_econ_crime)
df_violent_sexual_spark = spark.createDataFrame(df_violent_sexual)

ModuleNotFoundError: No module named 'distutils'